In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

Paths

In [2]:
train_dir = r"C:\Users\farna\OneDrive\Desktop\data science\dataset_dogs_vs_cats\train"
test_dir = r"C:\Users\farna\OneDrive\Desktop\data science\dataset_dogs_vs_cats\test"

 Image size and batch


In [3]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

Data augmentation

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 14874 images belonging to 2 classes.
Found 5023 images belonging to 2 classes.


Compute class weights to handle imbalance

In [6]:
y_train = train_data.classes
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

Base model

In [7]:
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze at first

Build the Full MOdel

In [8]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Callbacks

In [9]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


Initial training

In [10]:
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=5,
    class_weight=class_weights_dict,
    callbacks=[early_stop]
)


c:\Users\farna\miniconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 383s 815ms/step - accuracy: 0.9341 - loss: 0.1785 - val_accuracy: 0.9771 - val_loss: 0.0566
Epoch 2/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 350s 753ms/step - accuracy: 0.9700 - loss: 0.0749 - val_accuracy: 0.9751 - val_loss: 0.0639
Epoch 3/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 333s 717ms/step - accuracy: 0.9737 - loss: 0.0669 - val_accuracy: 0.9789 - val_loss: 0.0529
Epoch 4/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 339s 729ms/step - accuracy: 0.9761 - loss: 0.0649 - val_accuracy: 0.9695 - val_loss: 0.0815
Epoch 5/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 305s 656ms/step - accuracy: 0.9743 - loss: 0.0650 - val_accuracy: 0.9779 - val_loss: 0.0528


Fine-tuning


In [11]:
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Freeze all but top 20
    layer.trainable = False

Recompile after unfreezing

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Continue training

In [13]:
fine_tune_history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=5,
    class_weight=class_weights_dict,
    callbacks=[early_stop]
)

Epoch 1/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 362s 766ms/step - accuracy: 0.9450 - loss: 0.1489 - val_accuracy: 0.9737 - val_loss: 0.2835
Epoch 2/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 346s 745ms/step - accuracy: 0.9700 - loss: 0.0778 - val_accuracy: 0.9309 - val_loss: 0.3962
Epoch 3/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 370s 797ms/step - accuracy: 0.9747 - loss: 0.0626 - val_accuracy: 0.9707 - val_loss: 0.2064
Epoch 4/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 367s 790ms/step - accuracy: 0.9791 - loss: 0.0511 - val_accuracy: 0.9737 - val_loss: 0.1482
Epoch 5/5
465/465 ━━━━━━━━━━━━━━━━━━━━ 364s 783ms/step - accuracy: 0.9783 - loss: 0.0547 - val_accuracy: 0.9775 - val_loss: 0.1460


Evaluation

In [14]:
test_data.reset()
y_true = test_data.classes
y_pred = (model.predict(test_data) > 0.5).astype(int).reshape(-1)

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["cats", "dogs"]))


157/157 ━━━━━━━━━━━━━━━━━━━━ 69s 434ms/step

Confusion Matrix:
[[3129   48]
 [  65 1781]]

Classification Report:
              precision    recall  f1-score   support

        cats       0.98      0.98      0.98      3177
        dogs       0.97      0.96      0.97      1846

    accuracy                           0.98      5023
   macro avg       0.98      0.97      0.98      5023
weighted avg       0.98      0.98      0.98      5023

